# SageMaker Transform Batch Job

In [ ]:
import sagemaker
from sagemaker import Session
from sagemaker.model import Model
import boto3

# Define SageMaker session and client
sagemaker_session = Session()
sm_client = boto3.client("sagemaker")

# Existing SageMaker endpoint
endpoint_name = "TextClassificationMLDemo-TextClassification-Endpoint"

# Get model details from endpoint
response = sm_client.describe_endpoint_config(EndpointConfigName=endpoint_name)
model_name = response["ProductionVariants"][0]["ModelName"]

# Describe the model to get model data and container image
model_details = sm_client.describe_model(ModelName=model_name)
model_data = model_details["PrimaryContainer"]["ModelDataUrl"]
image_uri = model_details["PrimaryContainer"]["Image"]

# Define SageMaker Model
model = Model(
    model_data=model_data,
    image_uri=image_uri,
    role=sagemaker.get_execution_role(),
    sagemaker_session=sagemaker_session
)

# Create a transformer for batch transform
transformer = model.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://your-bucket-name/transform-output/',  # Change to your S3 bucket
    strategy='SingleRecord',
    assemble_with='Line',
    accept='application/json'
)

# Perform batch transformation
transformer.transform(
    data='s3://your-bucket-name/transform-input/',  # Change to your input data S3 path
    content_type='text/csv',
    split_type='Line',
    input_filter='$[0]',
    output_filter='$[1]',
    join_source='Input'
)

# Wait for transform job to complete
transformer.wait()

print("Batch transform job completed.")


In [ ]:
"ModelDockerImage-Desc": "SageMaker Prebuilt Docker Images: https://github.com/aws/deep-learning-containers/blob/master/available_images.md",